In [184]:
import json
import sqlite3
import os
import glob # For easily finding files with a specific pattern


json_root_dir = "C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\aggregated\\insurance\\country\\india"

# Define the path for your SQLite database
db_file_path = "ag_insurance_data.db" 

def setup_database(db_path):
    """Sets up the SQLite database tables."""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 1. Create the Transactions table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Transactions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                from_timestamp INTEGER,
                to_timestamp INTEGER,
                response_timestamp INTEGER,
                source_file TEXT -- Add a column to track which file this data came from
            )
        ''')

        # 2. Create the PaymentInstruments table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS PaymentInstruments (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                transaction_id INTEGER,
                type TEXT,
                count INTEGER,
                amount REAL,
                FOREIGN KEY (transaction_id) REFERENCES Transactions(id)
            )
        ''')
        conn.commit()
        print("Database tables created/ensured.")
    except sqlite3.Error as e:
        print(f"Error setting up database: {e}")
    finally:
        if conn:
            conn.close()

def insert_json_data(json_data, source_filename, db_path):
    """
    Inserts data from a single parsed JSON object into the SQLite database.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file_path)
        cursor = conn.cursor()

        # Extract top-level data
        success = json_data.get("success")
        code = json_data.get("code")
        response_timestamp = json_data.get("responseTimestamp")
        main_data = json_data.get("data", {})
        from_ts = main_data.get("from")
        to_ts = main_data.get("to")
        transaction_data_list = main_data.get("transactionData", [])

        # Insert into Transactions table
        if transaction_data_list:
            for transaction_item in transaction_data_list:
                transaction_name = transaction_item.get("name")

                cursor.execute('''
                    INSERT INTO Transactions (name, from_timestamp, to_timestamp, response_timestamp, source_file)
                    VALUES (?, ?, ?, ?, ?)
                ''', (transaction_name, from_ts, to_ts, response_timestamp, source_filename))
                transaction_id = cursor.lastrowid # Get the ID of the newly inserted row

                # Insert into PaymentInstruments table
                payment_instruments = transaction_item.get("paymentInstruments", [])
                for pi in payment_instruments:
                    pi_type = pi.get("type")
                    pi_count = pi.get("count")
                    pi_amount = pi.get("amount")

                    cursor.execute('''
                        INSERT INTO PaymentInstruments (transaction_id, type, count, amount)
                        VALUES (?, ?, ?, ?)
                    ''', (transaction_id, pi_type, pi_count, pi_amount))
            conn.commit()
            print(f"Successfully inserted data from '{source_filename}'.")
        else:
            print(f"No 'transactionData' found in '{source_filename}'. Skipping data insertion.")

    except sqlite3.Error as e:
        print(f"Error inserting data from '{source_filename}': {e}")
        if conn:
            conn.rollback() # Rollback changes for this file if an error occurs
    finally:
        if conn:
            conn.close()

def load_folder_of_jsons_to_sqlite(folder_path, db_path):
    """
    Loads all JSON files from a specified folder and its subfolders into a SQLite database.
    """
    setup_database(db_path) # Ensure tables are set up before starting insertions

    # Use glob to find all .json files in the folder and its subfolders
    # ** means look in subdirectories, * means any filename
    json_files = glob.glob(os.path.join(folder_path, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"No JSON files found in '{folder_path}' or its subdirectories.")
        return

    print(f"Found {len(json_files)} JSON files. Starting insertion...")
    for json_file in json_files:
        print(f"Processing: {json_file}")
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            # Pass the basename of the file for tracking
            insert_json_data(data, os.path.basename(json_file), db_path)
        except FileNotFoundError:
            print(f"Error: JSON file not found (should not happen here): {json_file}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {json_file}. Skipping.")
        except PermissionError:
            print(f"Permission denied for file: {json_file}. Skipping.")


# --- Main Execution ---
if __name__ == "__main__":
   
    # The glob pattern will find all .json files recursively from json_root_dir.

    load_folder_of_jsons_to_sqlite(json_root_dir, db_file_path)

    #Verify data
    print("\nVerifying data from all files...")
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM Transactions LIMIT 5") # Limit to 5 rows for brevity
    print("\nTransactions Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT * FROM PaymentInstruments LIMIT 5") # Limit to 5 rows for brevity
    print("\nPaymentInstruments Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    # Example: How many transactions from each file?
    cursor.execute("SELECT source_file, COUNT(*) FROM Transactions GROUP BY source_file")
    print("\nTransactions per source file:")
    for row in cursor.fetchall():
        print(row)


    conn.close()

Database tables created/ensured.
Found 703 JSON files. Starting insertion...
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\insurance\country\india\2020\2.json
Successfully inserted data from '2.json'.
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\insurance\country\india\2020\3.json
Successfully inserted data from '3.json'.
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\insurance\country\india\2020\4.json
Successfully inserted data from '4.json'.
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\insurance\country\india\2021\1.json
Successfully inserted data from '1.json'.
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\insurance\country\india\2021\2.json
Successfully inserted data from '2.json'.
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\insurance\country\india\2021\3.json
Successfully inserted data from '3.json'.
Proce

In [185]:

json_root_dir = "C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\aggregated\\transaction\\country\\india"


db_file_path = "ag_transaction_data.db" #filename to reflect multiple files

def setup_database(db_path):
    """Sets up the SQLite database tables."""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 1. Create the Transactions table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Transactions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                from_timestamp INTEGER,
                to_timestamp INTEGER,
                response_timestamp INTEGER,
                source_file TEXT -- Add a column to track which file this data came from
            )
        ''')

        # 2. Create the PaymentInstruments table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS PaymentInstruments (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                transaction_id INTEGER,
                type TEXT,
                count INTEGER,
                amount REAL,
                FOREIGN KEY (transaction_id) REFERENCES Transactions(id)
            )
        ''')
        conn.commit()
        print("Database tables created/ensured.")
    except sqlite3.Error as e:
        print(f"Error setting up database: {e}")
    finally:
        if conn:
            conn.close()

def insert_json_data(json_data, source_filename, db_path):
    """
    Inserts data from a single parsed JSON object into the SQLite database.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Extract top-level data
        success = json_data.get("success")
        code = json_data.get("code")
        response_timestamp = json_data.get("responseTimestamp")
        main_data = json_data.get("data", {})
        from_ts = main_data.get("from")
        to_ts = main_data.get("to")
        transaction_data_list = main_data.get("transactionData", [])

        # Insert into Transactions table
        if transaction_data_list:
            for transaction_item in transaction_data_list:
                transaction_name = transaction_item.get("name")

                cursor.execute('''
                    INSERT INTO Transactions (name, from_timestamp, to_timestamp, response_timestamp, source_file)
                    VALUES (?, ?, ?, ?, ?)
                ''', (transaction_name, from_ts, to_ts, response_timestamp, source_filename))
                transaction_id = cursor.lastrowid # Get the ID of the newly inserted row

                # Insert into PaymentInstruments table
                payment_instruments = transaction_item.get("paymentInstruments", [])
                for pi in payment_instruments:
                    pi_type = pi.get("type")
                    pi_count = pi.get("count")
                    pi_amount = pi.get("amount")

                    cursor.execute('''
                        INSERT INTO PaymentInstruments (transaction_id, type, count, amount)
                        VALUES (?, ?, ?, ?)
                    ''', (transaction_id, pi_type, pi_count, pi_amount))
            conn.commit()
            print(f"Successfully inserted data from '{source_filename}'.")
        else:
            print(f"No 'transactionData' found in '{source_filename}'. Skipping data insertion.")

    except sqlite3.Error as e:
        print(f"Error inserting data from '{source_filename}': {e}")
        if conn:
            conn.rollback() # Rollback changes for this file if an error occurs
    finally:
        if conn:
            conn.close()

def load_folder_of_jsons_to_sqlite(folder_path, db_path):
    """
    Loads all JSON files from a specified folder and its subfolders into a SQLite database.
    """
    setup_database(db_path) # Ensure tables are set up before starting insertions

    json_files = glob.glob(os.path.join(folder_path, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"No JSON files found in '{folder_path}' or its subdirectories.")
        return

    print(f"Found {len(json_files)} JSON files. Starting insertion...")
    for json_file in json_files:
        print(f"Processing: {json_file}")
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            # Pass the basename of the file for tracking
            insert_json_data(data, os.path.basename(json_file), db_path)
        except FileNotFoundError:
            print(f"Error: JSON file not found (should not happen here): {json_file}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {json_file}. Skipping.")
        except PermissionError:
            print(f"Permission denied for file: {json_file}. Skipping.")


# --- Main Execution ---
if __name__ == "__main__":
   
    # The glob pattern will find all .json files recursively from json_root_dir.

    load_folder_of_jsons_to_sqlite(json_root_dir, db_file_path)

    #Verify data
    print("\nVerifying data from all files...")
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM Transactions LIMIT 5") # Limit to 5 rows for brevity
    print("\nTransactions Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT * FROM PaymentInstruments LIMIT 5") # Limit to 5 rows for brevity
    print("\nPaymentInstruments Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    # Example: How many transactions from each file?
    cursor.execute("SELECT source_file, COUNT(*) FROM Transactions GROUP BY source_file")
    print("\nTransactions per source file:")
    for row in cursor.fetchall():
        print(row)


    conn.close()

Database tables created/ensured.
Found 1036 JSON files. Starting insertion...
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\transaction\country\india\2018\1.json
Successfully inserted data from '1.json'.
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\transaction\country\india\2018\2.json
Successfully inserted data from '2.json'.
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\transaction\country\india\2018\3.json
Successfully inserted data from '3.json'.
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\transaction\country\india\2018\4.json
Successfully inserted data from '4.json'.
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\transaction\country\india\2019\1.json
Successfully inserted data from '1.json'.
Processing: C:\Users\mukta\OneDrive\Desktop\Phonepay\pulse\data\aggregated\transaction\country\india\2019\2.json
Successfully inserted data from '2

In [186]:
import json

with open("C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\aggregated\\user\\country\\india\\2018\\1.json", "r") as f:
    data = json.load(f)

print(json.dumps(data, indent=4))

{
    "success": true,
    "code": "SUCCESS",
    "data": {
        "aggregated": {
            "registeredUsers": 46877867,
            "appOpens": 0
        },
        "usersByDevice": [
            {
                "brand": "Xiaomi",
                "count": 11926334,
                "percentage": 0.25441289809538475
            },
            {
                "brand": "Samsung",
                "count": 9609401,
                "percentage": 0.204988017052909
            },
            {
                "brand": "Vivo",
                "count": 5894293,
                "percentage": 0.1257372269092363
            },
            {
                "brand": "Oppo",
                "count": 4479351,
                "percentage": 0.09555364368434255
            },
            {
                "brand": "Realme",
                "count": 2376866,
                "percentage": 0.05070337351313361
            },
            {
                "brand": "Apple",
                "count": 182

In [187]:

import re # For regular expressions to extract year/month/day from path

json_root_dir = "C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\aggregated\\user\\country\\india" # <--- ADJUST THIS PATH

db_file_path = "ag_user_data.db" # <--- NEW DATABASE NAME

# --- Utility Function to Extract Date from Path ---
def extract_date_from_path(file_path):
    """
    Extracts year, month, and day from a file path using a common pattern like /YYYY/MM/DD.json
    Returns (year, month, day) or (None, None, None) if not found.
    """
   
    match = re.search(r'[\\/](\d{4})[\\/](\d{1,2})[\\/](\d{1,2})\.json$', file_path)
    if match:
        return int(match.group(1)), int(match.group(2)), int(match.group(3))
    
    # Fallback for paths like /YYYY/MM/some_id/filename.json if 1.json is just an ID
    match_ym = re.search(r'[\\/](\d{4})[\\/](\d{1,2})[\\/]', file_path)
    if match_ym:
        # If the filename itself is numeric, it might be the day
        day_match = re.search(r'[\\/](\d+)\.json$', file_path)
        day = int(day_match.group(1)) if day_match else None
        return int(match_ym.group(1)), int(match_ym.group(2)), day

    return None, None, None # Default if no pattern matches

# --- Database Setup Functions ---
def setup_database(db_path):
    """Sets up the SQLite database tables for aggregated user data."""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 1. Create the AggregatedSnapshots table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS AggregatedSnapshots (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                success INTEGER, -- SQLite uses 0 for False, 1 for True
                code TEXT,
                registeredUsers INTEGER,
                appOpens INTEGER,
                year INTEGER,
                month INTEGER,
                day INTEGER,
                source_file TEXT UNIQUE -- Assuming each JSON file represents a unique snapshot
            )
        ''')

        # 2. Create the UsersByDevice table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS UsersByDevice (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                snapshot_id INTEGER,
                brand TEXT,
                count INTEGER,
                percentage REAL,
                FOREIGN KEY (snapshot_id) REFERENCES AggregatedSnapshots(id)
            )
        ''')
        conn.commit()
        print("Database tables created/ensured for aggregated user data.")
    except sqlite3.Error as e:
        print(f"Error setting up database: {e}")
    finally:
        if conn:
            conn.close()

# --- Data Insertion Function ---
def insert_aggregated_user_snapshot(json_data, source_full_path, db_path):
    """
    Inserts data from a single parsed aggregated user JSON object into the SQLite database.
    """
    conn = None
    source_filename = os.path.basename(source_full_path)
    
    # Extract year, month, day from the full file path
    year, month, day = extract_date_from_path(source_full_path)

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Extract data from the JSON object
        success = 1 if json_data.get("success") else 0 # Convert boolean to integer for SQLite
        code = json_data.get("code")
        
        aggregated_data = json_data.get("data", {}).get("aggregated", {})
        registered_users = aggregated_data.get("registeredUsers")
        app_opens = aggregated_data.get("appOpens")
        
        # Insert into AggregatedSnapshots table
        # Using INSERT OR IGNORE to prevent re-inserting the same snapshot if the script is run multiple times
        cursor.execute('''
            INSERT OR IGNORE INTO AggregatedSnapshots (
                success, code, registeredUsers, appOpens, year, month, day, source_file
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            success, code, registered_users, app_opens, year, month, day, source_filename
        ))

        # If the snapshot was actually inserted (not ignored), get its ID
        if cursor.rowcount > 0: # rowcount will be 1 if inserted, 0 if ignored (due to UNIQUE source_file)
            snapshot_id = cursor.lastrowid
            
            # Insert into UsersByDevice table
            users_by_device_list = json_data.get("data", {}).get("usersByDevice", [])
            for device_data in users_by_device_list:
                brand = device_data.get("brand")
                count = device_data.get("count")
                percentage = device_data.get("percentage")

                cursor.execute('''
                    INSERT INTO UsersByDevice (snapshot_id, brand, count, percentage)
                    VALUES (?, ?, ?, ?)
                ''', (snapshot_id, brand, count, percentage))
            conn.commit()
            print(f"Successfully inserted data for snapshot '{source_filename}'.")
        else:
            print(f"Snapshot '{source_filename}' already exists in database. Skipping insertion.")

    except sqlite3.Error as e:
        print(f"Error inserting data from '{source_filename}': {e}")
        if conn:
            conn.rollback()
    except Exception as e: # Catch other potential errors during parsing/accessing
        print(f"General error processing '{source_filename}': {e}")
    finally:
        if conn:
            conn.close()

# --- Main Loading Logic ---
def load_folder_of_user_jsons_to_sqlite(folder_path, db_path):
    """
    Loads all JSON files from a specified folder and its subfolders (recursively)
    into a SQLite database for aggregated user data.
    """
    setup_database(db_path) # Ensure tables are set up before starting insertions

    # Use glob to find all .json files in the folder and its subfolders
    json_files = glob.glob(os.path.join(folder_path, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"No JSON files found in '{folder_path}' or its subdirectories.")
        return

    print(f"Found {len(json_files)} JSON files. Starting insertion...")
    for json_file in json_files:
        # print(f"Processing file: {json_file}") # Uncomment for more verbose output
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            insert_aggregated_user_snapshot(data, json_file, db_path) # Pass full path for date extraction
        except FileNotFoundError:
            print(f"Error: JSON file not found (should not happen here): {json_file}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {json_file}. Skipping.")
        except PermissionError:
            print(f"Permission denied for file: {json_file}. Skipping.")
        except Exception as e:
            print(f"An unexpected error occurred while processing {json_file}: {e}")


# --- Execution ---
if __name__ == "__main__":
   

    load_folder_of_user_jsons_to_sqlite(json_root_dir, db_file_path)

    # Optional: Verify data
    print("\n--- Verifying loaded data ---")
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    cursor.execute("SELECT id, registeredUsers, appOpens, year, month, day, source_file FROM AggregatedSnapshots LIMIT 5")
    print("\nAggregatedSnapshots Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT snapshot_id, brand, count, percentage FROM UsersByDevice LIMIT 5")
    print("\nUsersByDevice Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT source_file, COUNT(*) FROM AggregatedSnapshots GROUP BY source_file")
    print("\nSnapshots loaded per source file:")
    for row in cursor.fetchall():
        print(row)
    
    cursor.execute("SELECT year, month, COUNT(*) FROM AggregatedSnapshots GROUP BY year, month ORDER BY year, month")
    print("\nSnapshots per Year and Month:")
    for row in cursor.fetchall():
        print(row)

    conn.close()

Database tables created/ensured for aggregated user data.
Found 1036 JSON files. Starting insertion...
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in dat

In [171]:
import json

with open("C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\map\\insurance\\country\\india\\2020\\2.json", "r") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

{
    "success": true,
    "code": "SUCCESS",
    "data": {
        "meta": {
            "dataLevel": "COUNTRY",
            "gridLevel": 10,
            "percentiles": {
                "10.0": 1,
                "20.0": 1,
                "40.0": 1,
                "80.0": 7,
                "90.0": 17,
                "99.5": 563,
                "50.0": 2,
                "30.0": 1,
                "60.0": 3
            }
        },
        "data": {
            "columns": [
                "lat",
                "lng",
                "metric",
                "label"
            ],
            "data": [
                [
                    12.88117483333963,
                    77.56767350389504,
                    4720.0,
                    "karnataka"
                ],
                [
                    17.42819734031383,
                    78.38991078921848,
                    3186.0,
                    "telangana"
                ],
                [
              

In [172]:

json_root_dir = "C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\map\\insurance\\country\\india" # <--- ADJUST THIS PATH

# Define the path for your SQLite database file
db_file_path = "map_insurance_data.db" # <--- NEW DATABASE NAME

# --- Utility Function to Extract Date/Period from Path ---
def extract_period_from_path(file_path):
    """
    Extracts year and quarter/month/id from a file path using a common pattern like /YYYY/QQ.json or /YYYY/MM.json
    Returns (year, period) or (None, None) if not found.
    """
    # Regex to find /YYYY/NUMBER.json pattern
    match = re.search(r'[\\/](\d{4})[\\/](\d{1,2})\.json$', file_path)
    if match:
        return int(match.group(1)), int(match.group(2))
    
    # Fallback if filename is not a direct number, but the structure is /YYYY/MM/filename.json
    match_ym = re.search(r'[\\/](\d{4})[\\/](\d{1,2})[\\/]', file_path)
    if match_ym:
        # If the filename itself is numeric, it might be the day
        period_match = re.search(r'[\\/](\d+)\.json$', file_path)
        period = int(period_match.group(1)) if period_match else None
        return int(match_ym.group(1)), period

    return None, None # Default if no pattern matches

# --- Database Setup Functions ---
def setup_database(db_path):
    """Sets up the SQLite database tables for map insurance data."""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 1. Create the MapInsuranceSnapshots table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS MapInsuranceSnapshots (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                success INTEGER, -- SQLite uses 0 for False, 1 for True
                code TEXT,
                response_timestamp INTEGER,
                year INTEGER,
                quarter_or_month INTEGER,
                source_file TEXT UNIQUE
            )
        ''')

        # 2. Create the StateMetrics table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS StateMetrics (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                snapshot_id INTEGER,
                state_name TEXT,
                total_count INTEGER,
                total_amount REAL,
                FOREIGN KEY (snapshot_id) REFERENCES MapInsuranceSnapshots(id)
            )
        ''')

        # 3. Create the DistrictMetrics table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS DistrictMetrics (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                state_metric_id INTEGER,
                district_name TEXT,
                district_count INTEGER,
                district_amount REAL,
                FOREIGN KEY (state_metric_id) REFERENCES StateMetrics(id)
            )
        ''')
        conn.commit()
        print("Database tables created/ensured for map insurance data.")
    except sqlite3.Error as e:
        print(f"Error setting up database: {e}")
    finally:
        if conn:
            conn.close()

# --- Data Insertion Function ---
def insert_map_insurance_snapshot(json_data, source_full_path, db_path):
    """
    Inserts data from a single parsed map insurance JSON object into the SQLite database.
    """
    conn = None
    source_filename = os.path.basename(source_full_path)
    
    # Extract year and quarter/month from the full file path
    year, quarter_or_month = extract_period_from_path(source_full_path)

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Extract data from the JSON object
        success = 1 if json_data.get("success") else 0 # Convert boolean to integer
        code = json_data.get("code")
        response_timestamp = json_data.get("responseTimestamp")
        
        # Insert into MapInsuranceSnapshots table
        cursor.execute('''
            INSERT OR IGNORE INTO MapInsuranceSnapshots (
                success, code, response_timestamp, year, quarter_or_month, source_file
            )
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            success, code, response_timestamp, year, quarter_or_month, source_filename
        ))

        # If the snapshot was actually inserted, get its ID
        if cursor.rowcount > 0:
            snapshot_id = cursor.lastrowid
            
            # Process hoverData (states/union territories)
            hover_data = json_data.get("data", {}).get("hoverData", {})
            for state_name, state_data in hover_data.items():
                state_metric = state_data.get("metric", {})
                state_count = state_metric.get("count")
                state_amount = state_metric.get("amount")

                cursor.execute('''
                    INSERT INTO StateMetrics (snapshot_id, state_name, total_count, total_amount)
                    VALUES (?, ?, ?, ?)
                ''', (snapshot_id, state_name, state_count, state_amount))
                state_metric_id = cursor.lastrowid # Get ID for linking districts

                # Process districtData for the current state
                district_data_list = state_data.get("districtData", [])
                for district_item in district_data_list:
                    district_name = district_item.get("name")
                    district_metric = district_item.get("metric", {})
                    district_count = district_metric.get("count")
                    district_amount = district_metric.get("amount")

                    cursor.execute('''
                        INSERT INTO DistrictMetrics (state_metric_id, district_name, district_count, district_amount)
                        VALUES (?, ?, ?, ?)
                    ''', (state_metric_id, district_name, district_count, district_amount))
            conn.commit()
            print(f"Successfully inserted data for snapshot '{source_filename}'.")
        else:
            print(f"Snapshot '{source_filename}' already exists in database. Skipping insertion.")

    except sqlite3.Error as e:
        print(f"Error inserting data from '{source_filename}': {e}")
        if conn:
            conn.rollback()
    except Exception as e: # Catch other potential errors during parsing/accessing
        print(f"General error processing '{source_filename}': {e}")
    finally:
        if conn:
            conn.close()

# --- Main Loading Logic ---
def load_folder_of_map_jsons_to_sqlite(folder_path, db_path):
    """
    Loads all JSON files from a specified folder and its subfolders (recursively)
    into a SQLite database for map insurance data.
    """
    setup_database(db_path) # Ensure tables are set up before starting insertions

    # Use glob to find all .json files in the folder and its subfolders
    json_files = glob.glob(os.path.join(folder_path, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"No JSON files found in '{folder_path}' or its subdirectories.")
        return

    print(f"Found {len(json_files)} JSON files. Starting insertion...")
    for json_file in json_files:
        # print(f"Processing file: {json_file}") # Uncomment for more verbose output
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            insert_map_insurance_snapshot(data, json_file, db_path) # Pass full path for date extraction
        except FileNotFoundError:
            print(f"Error: JSON file not found (should not happen here): {json_file}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {json_file}. Skipping.")
        except PermissionError:
            print(f"Permission denied for file: {json_file}. Skipping.")
        except Exception as e:
            print(f"An unexpected error occurred while processing {json_file}: {e}")


# --- Execution ---
if __name__ == "__main__":
    

    load_folder_of_map_jsons_to_sqlite(json_root_dir, db_file_path)

    #Verify data
    print("\n--- Verifying loaded data ---")
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    cursor.execute("SELECT id, success, code, year, quarter_or_month, source_file FROM MapInsuranceSnapshots LIMIT 5")
    print("\nMapInsuranceSnapshots Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT snapshot_id, state_name, total_count, total_amount FROM StateMetrics LIMIT 5")
    print("\nStateMetrics Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT state_metric_id, district_name, district_count, district_amount FROM DistrictMetrics LIMIT 5")
    print("\nDistrictMetrics Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT source_file, COUNT(*) FROM MapInsuranceSnapshots GROUP BY source_file")
    print("\nSnapshots loaded per source file:")
    for row in cursor.fetchall():
        print(row)
    
    cursor.execute("SELECT year, quarter_or_month, COUNT(*) FROM MapInsuranceSnapshots GROUP BY year, quarter_or_month ORDER BY year, quarter_or_month")
    print("\nSnapshots per Year and Quarter/Month:")
    for row in cursor.fetchall():
        print(row)

    conn.close()

Database tables created/ensured for map insurance data.
Found 701 JSON files. Starting insertion...
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in databa

In [173]:

json_root_dir = "C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\map\\insurance\\hover\\country\\india" 

# Define the path for your SQLite database file
db_file_path = "map_hover_data.db" # <--- NEW DATABASE NAME

# --- Utility Function to Extract Date/Period from Path ---
def extract_period_from_path(file_path):
    """
    Extracts year and a generic period ID from a file path using a common pattern like /YYYY/PERIOD.json
    Returns (year, period) or (None, None) if not found.
    """
    # Regex to find /YYYY/NUMBER.json pattern
    match = re.search(r'[\\/](\d{4})[\\/](\d{1,3})\.json$', file_path)
    if match:
        return int(match.group(1)), int(match.group(2))
    
    return None, None # Default if no pattern matches

# --- Database Setup Functions ---
def setup_database(db_path):
    """Sets up the SQLite database tables for map hover data."""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 1. Create the MapHoverSnapshots table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS MapHoverSnapshots (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                success INTEGER, -- SQLite uses 0 for False, 1 for True
                code TEXT,
                year INTEGER,
                period_id INTEGER, -- Month, quarter, or arbitrary ID
                source_file TEXT UNIQUE
            )
        ''')

        # 2. Create the HoverDataEntries table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS HoverDataEntries (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                snapshot_id INTEGER,
                region_name TEXT,
                metric_type TEXT,
                metric_count INTEGER,
                metric_amount REAL,
                FOREIGN KEY (snapshot_id) REFERENCES MapHoverSnapshots(id)
            )
        ''')

        conn.commit()
        print("Database tables created/ensured for map hover data.")
    except sqlite3.Error as e:
        print(f"Error setting up database: {e}")
    finally:
        if conn:
            conn.close()

# --- Data Insertion Function ---
def insert_map_hover_snapshot(json_data, source_full_path, db_path):
    """
    Inserts data from a single parsed map hover JSON object into the SQLite database.
    """
    conn = None
    source_filename = os.path.basename(source_full_path)
    
    # Extract year and period_id from the full file path
    year, period_id = extract_period_from_path(source_full_path)

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Extract data from the JSON object
        success = 1 if json_data.get("success") else 0 # Convert boolean to integer
        code = json_data.get("code")
        
        # Insert into MapHoverSnapshots table
        cursor.execute('''
            INSERT OR IGNORE INTO MapHoverSnapshots (
                success, code, year, period_id, source_file
            )
            VALUES (?, ?, ?, ?, ?)
        ''', (
            success, code, year, period_id, source_filename
        ))

        # If the snapshot was actually inserted, get its ID
        if cursor.rowcount > 0:
            snapshot_id = cursor.lastrowid
            
            # Process hoverdatalist
            hover_data_list = json_data.get("data", {}).get("hoverdatalist", [])
            for entry in hover_data_list:
                name = entry.get("name")
                metric = entry.get("metric", {})
                metric_type = metric.get("type")
                metric_count = metric.get("count")
                metric_amount = metric.get("amount")

                cursor.execute('''
                    INSERT INTO HoverDataEntries (snapshot_id, region_name, metric_type, metric_count, metric_amount)
                    VALUES (?, ?, ?, ?, ?)
                ''', (snapshot_id, name, metric_type, metric_count, metric_amount))
            conn.commit()
            print(f"Successfully inserted data for snapshot '{source_filename}'.")
        else:
            print(f"Snapshot '{source_filename}' already exists in database. Skipping insertion.")

    except sqlite3.Error as e:
        print(f"Error inserting data from '{source_filename}': {e}")
        if conn:
            conn.rollback()
    except Exception as e: # Catch other potential errors during parsing/accessing
        print(f"General error processing '{source_filename}': {e}")
    finally:
        if conn:
            conn.close()

# --- Main Loading Logic ---
def load_folder_of_map_hover_jsons_to_sqlite(folder_path, db_path):
    """
    Loads all JSON files from a specified folder and its subfolders (recursively)
    into a SQLite database for map hover data.
    """
    setup_database(db_path) # Ensure tables are set up before starting insertions

    # Use glob to find all .json files in the folder and its subfolders
    json_files = glob.glob(os.path.join(folder_path, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"No JSON files found in '{folder_path}' or its subdirectories.")
        return

    print(f"Found {len(json_files)} JSON files. Starting insertion...")
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            insert_map_hover_snapshot(data, json_file, db_path) # Pass full path for date extraction
        except FileNotFoundError:
            print(f"Error: JSON file not found (should not happen here): {json_file}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {json_file}. Skipping.")
        except PermissionError:
            print(f"Permission denied for file: {json_file}. Skipping.")
        except Exception as e:
            print(f"An unexpected error occurred while processing {json_file}: {e}")


# --- Execution ---
if __name__ == "__main__":
   

    load_folder_of_map_hover_jsons_to_sqlite(json_root_dir, db_file_path)

    #Verify data
    print("\n--- Verifying loaded data ---")
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    cursor.execute("SELECT id, success, code, year, period_id, source_file FROM MapHoverSnapshots LIMIT 5")
    print("\nMapHoverSnapshots Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT snapshot_id, region_name, metric_type, metric_count, metric_amount FROM HoverDataEntries LIMIT 5")
    print("\nHoverDataEntries Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT source_file, COUNT(*) FROM MapHoverSnapshots GROUP BY source_file")
    print("\nSnapshots loaded per source file:")
    for row in cursor.fetchall():
        print(row)
    
    cursor.execute("SELECT year, period_id, COUNT(*) FROM MapHoverSnapshots GROUP BY year, period_id ORDER BY year, period_id")
    print("\nSnapshots per Year and Period:")
    for row in cursor.fetchall():
        print(row)

    conn.close()

Database tables created/ensured for map hover data.
Found 703 JSON files. Starting insertion...
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. 

In [174]:
import json

with open("C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\map\\insurance\\hover\\country\\india\\2020\\2.json", "r") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

{
    "success": true,
    "code": "SUCCESS",
    "data": {
        "hoverDataList": [
            {
                "name": "puducherry",
                "metric": [
                    {
                        "type": "TOTAL",
                        "count": 112,
                        "amount": 22251.0
                    }
                ]
            },
            {
                "name": "tamil nadu",
                "metric": [
                    {
                        "type": "TOTAL",
                        "count": 5473,
                        "amount": 1075552.0
                    }
                ]
            },
            {
                "name": "uttar pradesh",
                "metric": [
                    {
                        "type": "TOTAL",
                        "count": 9884,
                        "amount": 1912266.0
                    }
                ]
            },
            {
                "name": "madhya pradesh",
              

In [175]:

json_root_dir = "C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\map\\transaction\\hover\\country\india" 

# Define the path for your SQLite database file
db_file_path = "map_transaction_data.db" # <--- NEW DATABASE NAME

# --- Utility Function to Extract Date/Period from Path ---
def extract_period_from_path(file_path):
    """
    Extracts year and a generic period ID from a file path using a common pattern like /YYYY/PERIOD.json
    Returns (year, period) or (None, None) if not found.
    """
    # Regex to find /YYYY/NUMBER.json pattern
    match = re.search(r'[\\/](\d{4})[\\/](\d{1,3})\.json$', file_path)
    if match:
        return int(match.group(1)), int(match.group(2))
    
    return None, None # Default if no pattern matches

# --- Database Setup Functions ---
def setup_database(db_path):
    """Sets up the SQLite database tables for map hover data."""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 1. Create the MapHoverSnapshots table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS MapHoverSnapshots (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                success INTEGER, -- SQLite uses 0 for False, 1 for True
                code TEXT,
                year INTEGER,
                period_id INTEGER, -- Month, quarter, or arbitrary ID
                source_file TEXT UNIQUE
            )
        ''')

        # 2. Create the HoverDataEntries table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS HoverDataEntries (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                snapshot_id INTEGER,
                region_name TEXT,
                metric_type TEXT,
                metric_count INTEGER,
                metric_amount REAL,
                FOREIGN KEY (snapshot_id) REFERENCES MapHoverSnapshots(id)
            )
        ''')

        conn.commit()
        print("Database tables created/ensured for map hover data.")
    except sqlite3.Error as e:
        print(f"Error setting up database: {e}")
    finally:
        if conn:
            conn.close()

# --- Data Insertion Function ---
def insert_map_hover_snapshot(json_data, source_full_path, db_path):
    """
    Inserts data from a single parsed map hover JSON object into the SQLite database.
    """
    conn = None
    source_filename = os.path.basename(source_full_path)
    
    # Extract year and period_id from the full file path
    year, period_id = extract_period_from_path(source_full_path)

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Extract data from the JSON object
        success = 1 if json_data.get("success") else 0 # Convert boolean to integer
        code = json_data.get("code")
        
        # Insert into MapHoverSnapshots table
        cursor.execute('''
            INSERT OR IGNORE INTO MapHoverSnapshots (
                success, code, year, period_id, source_file
            )
            VALUES (?, ?, ?, ?, ?)
        ''', (
            success, code, year, period_id, source_filename
        ))

        # If the snapshot was actually inserted, get its ID
        if cursor.rowcount > 0:
            snapshot_id = cursor.lastrowid
            
            # Process hoverdatalist
            hover_data_list = json_data.get("data", {}).get("hoverdatalist", [])
            for entry in hover_data_list:
                name = entry.get("name")
                metric = entry.get("metric", {})
                metric_type = metric.get("type")
                metric_count = metric.get("count")
                metric_amount = metric.get("amount")

                cursor.execute('''
                    INSERT INTO HoverDataEntries (snapshot_id, region_name, metric_type, metric_count, metric_amount)
                    VALUES (?, ?, ?, ?, ?)
                ''', (snapshot_id, name, metric_type, metric_count, metric_amount))
            conn.commit()
            print(f"Successfully inserted data for snapshot '{source_filename}'.")
        else:
            print(f"Snapshot '{source_filename}' already exists in database. Skipping insertion.")

    except sqlite3.Error as e:
        print(f"Error inserting data from '{source_filename}': {e}")
        if conn:
            conn.rollback()
    except Exception as e: # Catch other potential errors during parsing/accessing
        print(f"General error processing '{source_filename}': {e}")
    finally:
        if conn:
            conn.close()

# --- Main Loading Logic ---
def load_folder_of_map_hover_jsons_to_sqlite(folder_path, db_path):
    """
    Loads all JSON files from a specified folder and its subfolders (recursively)
    into a SQLite database for map hover data.
    """
    setup_database(db_path) # Ensure tables are set up before starting insertions

    # Use glob to find all .json files in the folder and its subfolders
    json_files = glob.glob(os.path.join(folder_path, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"No JSON files found in '{folder_path}' or its subdirectories.")
        return

    print(f"Found {len(json_files)} JSON files. Starting insertion...")
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            insert_map_hover_snapshot(data, json_file, db_path) # Pass full path for date extraction
        except FileNotFoundError:
            print(f"Error: JSON file not found (should not happen here): {json_file}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {json_file}. Skipping.")
        except PermissionError:
            print(f"Permission denied for file: {json_file}. Skipping.")
        except Exception as e:
            print(f"An unexpected error occurred while processing {json_file}: {e}")


# --- Execution ---
if __name__ == "__main__":
   

    load_folder_of_map_hover_jsons_to_sqlite(json_root_dir, db_file_path)

    # Verify data
    print("\n--- Verifying loaded data ---")
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    cursor.execute("SELECT id, success, code, year, period_id, source_file FROM MapHoverSnapshots LIMIT 5")
    print("\nMapHoverSnapshots Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT snapshot_id, region_name, metric_type, metric_count, metric_amount FROM HoverDataEntries LIMIT 5")
    print("\nHoverDataEntries Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT source_file, COUNT(*) FROM MapHoverSnapshots GROUP BY source_file")
    print("\nSnapshots loaded per source file:")
    for row in cursor.fetchall():
        print(row)
    
    cursor.execute("SELECT year, period_id, COUNT(*) FROM MapHoverSnapshots GROUP BY year, period_id ORDER BY year, period_id")
    print("\nSnapshots per Year and Period:")
    for row in cursor.fetchall():
        print(row)

    conn.close()

Database tables created/ensured for map hover data.
Found 1036 JSON files. Starting insertion...
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.


<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
C:\Users\mukta\AppData\Local\Temp\ipykernel_23436\2948765.py:1: SyntaxWarning: invalid escape sequence '\i'
  json_root_dir = "C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\map\\transaction\\hover\\country\india"


Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '

In [176]:

json_root_dir = "C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\map\\user\\hover\\country\\india" 

# Define the path for your SQLite database file
db_file_path = "map_user_data.db" # <--- NEW DATABASE NAME

# --- Utility Function to Extract Date/Period from Path ---
def extract_period_from_path(file_path):
    """
    Extracts year and a generic period ID from a file path using a common pattern like /YYYY/PERIOD.json
    Returns (year, period) or (None, None) if not found.
    """
    # Regex to find /YYYY/NUMBER.json pattern
    match = re.search(r'[\\/](\d{4})[\\/](\d{1,3})\.json$', file_path)
    if match:
        return int(match.group(1)), int(match.group(2))
    
    return None, None # Default if no pattern matches

# --- Database Setup Functions ---
def setup_database(db_path):
    """Sets up the SQLite database tables for map hover data."""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 1. Create the MapHoverSnapshots table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS MapHoverSnapshots (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                success INTEGER, -- SQLite uses 0 for False, 1 for True
                code TEXT,
                year INTEGER,
                period_id INTEGER, -- Month, quarter, or arbitrary ID
                source_file TEXT UNIQUE
            )
        ''')

        # 2. Create the HoverDataEntries table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS HoverDataEntries (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                snapshot_id INTEGER,
                region_name TEXT,
                metric_type TEXT,
                metric_count INTEGER,
                metric_amount REAL,
                FOREIGN KEY (snapshot_id) REFERENCES MapHoverSnapshots(id)
            )
        ''')

        conn.commit()
        print("Database tables created/ensured for map hover data.")
    except sqlite3.Error as e:
        print(f"Error setting up database: {e}")
    finally:
        if conn:
            conn.close()

# --- Data Insertion Function ---
def insert_map_hover_snapshot(json_data, source_full_path, db_path):
    """
    Inserts data from a single parsed map hover JSON object into the SQLite database.
    """
    conn = None
    source_filename = os.path.basename(source_full_path)
    
    # Extract year and period_id from the full file path
    year, period_id = extract_period_from_path(source_full_path)

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Extract data from the JSON object
        success = 1 if json_data.get("success") else 0 # Convert boolean to integer
        code = json_data.get("code")
        
        # Insert into MapHoverSnapshots table
        cursor.execute('''
            INSERT OR IGNORE INTO MapHoverSnapshots (
                success, code, year, period_id, source_file
            )
            VALUES (?, ?, ?, ?, ?)
        ''', (
            success, code, year, period_id, source_filename
        ))

        # If the snapshot was actually inserted, get its ID
        if cursor.rowcount > 0:
            snapshot_id = cursor.lastrowid
            
            # Process hoverdatalist
            hover_data_list = json_data.get("data", {}).get("hoverdatalist", [])
            for entry in hover_data_list:
                name = entry.get("name")
                metric = entry.get("metric", {})
                metric_type = metric.get("type")
                metric_count = metric.get("count")
                metric_amount = metric.get("amount")

                cursor.execute('''
                    INSERT INTO HoverDataEntries (snapshot_id, region_name, metric_type, metric_count, metric_amount)
                    VALUES (?, ?, ?, ?, ?)
                ''', (snapshot_id, name, metric_type, metric_count, metric_amount))
            conn.commit()
            print(f"Successfully inserted data for snapshot '{source_filename}'.")
        else:
            print(f"Snapshot '{source_filename}' already exists in database. Skipping insertion.")

    except sqlite3.Error as e:
        print(f"Error inserting data from '{source_filename}': {e}")
        if conn:
            conn.rollback()
    except Exception as e: # Catch other potential errors during parsing/accessing
        print(f"General error processing '{source_filename}': {e}")
    finally:
        if conn:
            conn.close()

# --- Main Loading Logic ---
def load_folder_of_map_hover_jsons_to_sqlite(folder_path, db_path):
    """
    Loads all JSON files from a specified folder and its subfolders (recursively)
    into a SQLite database for map hover data.
    """
    setup_database(db_path) # Ensure tables are set up before starting insertions

    # Use glob to find all .json files in the folder and its subfolders
    json_files = glob.glob(os.path.join(folder_path, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"No JSON files found in '{folder_path}' or its subdirectories.")
        return

    print(f"Found {len(json_files)} JSON files. Starting insertion...")
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            insert_map_hover_snapshot(data, json_file, db_path) # Pass full path for date extraction
        except FileNotFoundError:
            print(f"Error: JSON file not found (should not happen here): {json_file}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {json_file}. Skipping.")
        except PermissionError:
            print(f"Permission denied for file: {json_file}. Skipping.")
        except Exception as e:
            print(f"An unexpected error occurred while processing {json_file}: {e}")


# --- Execution ---
if __name__ == "__main__":
    
    load_folder_of_map_hover_jsons_to_sqlite(json_root_dir, db_file_path)

    # Verify data
    print("\n--- Verifying loaded data ---")
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    cursor.execute("SELECT id, success, code, year, period_id, source_file FROM MapHoverSnapshots LIMIT 5")
    print("\nMapHoverSnapshots Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT snapshot_id, region_name, metric_type, metric_count, metric_amount FROM HoverDataEntries LIMIT 5")
    print("\nHoverDataEntries Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT source_file, COUNT(*) FROM MapHoverSnapshots GROUP BY source_file")
    print("\nSnapshots loaded per source file:")
    for row in cursor.fetchall():
        print(row)
    
    cursor.execute("SELECT year, period_id, COUNT(*) FROM MapHoverSnapshots GROUP BY year, period_id ORDER BY year, period_id")
    print("\nSnapshots per Year and Period:")
    for row in cursor.fetchall():
        print(row)

    conn.close()

Database tables created/ensured for map hover data.
Found 1036 JSON files. Starting insertion...
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database.

In [177]:
import json

with open("C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\top\\insurance\\country\\india\\2020\\2.json", "r") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

{
    "success": true,
    "code": "SUCCESS",
    "data": {
        "states": [
            {
                "entityName": "maharashtra",
                "metric": {
                    "type": "TOTAL",
                    "count": 39836,
                    "amount": 6879717.0
                }
            },
            {
                "entityName": "karnataka",
                "metric": {
                    "type": "TOTAL",
                    "count": 27358,
                    "amount": 4794150.0
                }
            },
            {
                "entityName": "andhra pradesh",
                "metric": {
                    "type": "TOTAL",
                    "count": 22104,
                    "amount": 3982391.0
                }
            },
            {
                "entityName": "telangana",
                "metric": {
                    "type": "TOTAL",
                    "count": 19003,
                    "amount": 3419453.0
                }
    

In [178]:

json_root_dir = "C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\top\\insurance\\country\\india" # <--- ADJUST THIS PATH

# Define the path for your SQLite database file
db_file_path = "top_insurance_data.db" # <--- NEW DATABASE NAME

# --- Utility Function to Extract Date/Period from Path ---
def extract_period_from_path(file_path):
    """
    Extracts year and a generic period ID from a file path using a common pattern like /YYYY/PERIOD.json
    Returns (year, period) or (None, None) if not found.
    """
    # Regex to find /YYYY/NUMBER.json pattern
    match = re.search(r'[\\/](\d{4})[\\/](\d{1,3})\.json$', file_path)
    if match:
        return int(match.group(1)), int(match.group(2))
    
    return None, None # Default if no pattern matches

# --- Database Setup Functions ---
def setup_database(db_path):
    """Sets up the SQLite database tables for top insurance data."""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 1. Create the TopInsuranceSnapshots table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS TopInsuranceSnapshots (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                success INTEGER, -- SQLite uses 0 for False, 1 for True
                code TEXT,
                year INTEGER,
                period_id INTEGER, -- Month, quarter, or arbitrary ID
                source_file TEXT UNIQUE
            )
        ''')

        # 2. Create the TopInsuranceEntities table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS TopInsuranceEntities (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                snapshot_id INTEGER,
                entity_name TEXT,
                metric_type TEXT,
                metric_count INTEGER,
                metric_amount REAL,
                FOREIGN KEY (snapshot_id) REFERENCES TopInsuranceSnapshots(id)
            )
        ''')

        conn.commit()
        print("Database tables created/ensured for top insurance data.")
    except sqlite3.Error as e:
        print(f"Error setting up database: {e}")
    finally:
        if conn:
            conn.close()

# --- Data Insertion Function ---
def insert_top_insurance_snapshot(json_data, source_full_path, db_path):
    """
    Inserts data from a single parsed top insurance JSON object into the SQLite database.
    """
    conn = None
    source_filename = os.path.basename(source_full_path)
    
    # Extract year and period_id from the full file path
    year, period_id = extract_period_from_path(source_full_path)

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Extract data from the JSON object
        success = 1 if json_data.get("success") else 0 # Convert boolean to integer
        code = json_data.get("code")
        
        # Insert into TopInsuranceSnapshots table
        cursor.execute('''
            INSERT OR IGNORE INTO TopInsuranceSnapshots (
                success, code, year, period_id, source_file
            )
            VALUES (?, ?, ?, ?, ?)
        ''', (
            success, code, year, period_id, source_filename
        ))

        # If the snapshot was actually inserted, get its ID
        if cursor.rowcount > 0:
            snapshot_id = cursor.lastrowid
            
            # Process 'states' list
            states_list = json_data.get("data", {}).get("states", [])
            for entity_data in states_list:
                entity_name = entity_data.get("entityName")
                metric = entity_data.get("metric", {})
                metric_type = metric.get("type")
                metric_count = metric.get("count")
                metric_amount = metric.get("amount")

                cursor.execute('''
                    INSERT INTO TopInsuranceEntities (snapshot_id, entity_name, metric_type, metric_count, metric_amount)
                    VALUES (?, ?, ?, ?, ?)
                ''', (snapshot_id, entity_name, metric_type, metric_count, metric_amount))
            conn.commit()
            print(f"Successfully inserted data for snapshot '{source_filename}'.")
        else:
            print(f"Snapshot '{source_filename}' already exists in database. Skipping insertion.")

    except sqlite3.Error as e:
        print(f"Error inserting data from '{source_filename}': {e}")
        if conn:
            conn.rollback()
    except Exception as e: # Catch other potential errors during parsing/accessing
        print(f"General error processing '{source_filename}': {e}")
    finally:
        if conn:
            conn.close()

# --- Main Loading Logic ---
def load_folder_of_top_insurance_jsons_to_sqlite(folder_path, db_path):
    """
    Loads all JSON files from a specified folder and its subfolders (recursively)
    into a SQLite database for top insurance data.
    """
    setup_database(db_path) # Ensure tables are set up before starting insertions

    # Use glob to find all .json files in the folder and its subfolders
    json_files = glob.glob(os.path.join(folder_path, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"No JSON files found in '{folder_path}' or its subdirectories.")
        return

    print(f"Found {len(json_files)} JSON files. Starting insertion...")
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            insert_top_insurance_snapshot(data, json_file, db_path) # Pass full path for date extraction
        except FileNotFoundError:
            print(f"Error: JSON file not found (should not happen here): {json_file}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {json_file}. Skipping.")
        except PermissionError:
            print(f"Permission denied for file: {json_file}. Skipping.")
        except Exception as e:
            print(f"An unexpected error occurred while processing {json_file}: {e}")


# --- Execution ---
if __name__ == "__main__":
  

    load_folder_of_top_insurance_jsons_to_sqlite(json_root_dir, db_file_path)

    # Optional: Verify data
    print("\n--- Verifying loaded data ---")
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    cursor.execute("SELECT id, success, code, year, period_id, source_file FROM TopInsuranceSnapshots LIMIT 5")
    print("\nTopInsuranceSnapshots Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT snapshot_id, entity_name, metric_type, metric_count, metric_amount FROM TopInsuranceEntities LIMIT 5")
    print("\nTopInsuranceEntities Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT source_file, COUNT(*) FROM TopInsuranceSnapshots GROUP BY source_file")
    print("\nSnapshots loaded per source file:")
    for row in cursor.fetchall():
        print(row)
    
    cursor.execute("SELECT year, period_id, COUNT(*) FROM TopInsuranceSnapshots GROUP BY year, period_id ORDER BY year, period_id")
    print("\nSnapshots per Year and Period:")
    for row in cursor.fetchall():
        print(row)

    conn.close()

Database tables created/ensured for top insurance data.
Found 703 JSON files. Starting insertion...
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in databa

In [179]:
import json

with open("C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\top\\transaction\\country\\india\\2018\\1.json", "r") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

{
    "success": true,
    "code": "SUCCESS",
    "data": {
        "states": [
            {
                "entityName": "maharashtra",
                "metric": {
                    "type": "TOTAL",
                    "count": 16387034,
                    "amount": 21711613257.725674
                }
            },
            {
                "entityName": "uttar pradesh",
                "metric": {
                    "type": "TOTAL",
                    "count": 12537805,
                    "amount": 13939970923.223106
                }
            },
            {
                "entityName": "karnataka",
                "metric": {
                    "type": "TOTAL",
                    "count": 12016899,
                    "amount": 19217895314.012802
                }
            },
            {
                "entityName": "west bengal",
                "metric": {
                    "type": "TOTAL",
                    "count": 11710225,
                    "a

In [180]:

json_root_dir = "C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\top\\transaction\\country\\india" # <--- ADJUST THIS PATH

# Define the path for your SQLite database file
db_file_path = "top_transaction_data.db" # <--- NEW DATABASE NAME

# --- Utility Function to Extract Date/Period from Path ---
def extract_period_from_path(file_path):
    """
    Extracts year and a generic period ID from a file path using a common pattern like /YYYY/PERIOD.json
    Returns (year, period) or (None, None) if not found.
    """
    # Regex to find /YYYY/NUMBER.json pattern
    match = re.search(r'[\\/](\d{4})[\\/](\d{1,3})\.json$', file_path)
    if match:
        return int(match.group(1)), int(match.group(2))
    
    return None, None # Default if no pattern matches

# --- Database Setup Functions ---
def setup_database(db_path):
    """Sets up the SQLite database tables for top insurance data."""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 1. Create the TopInsuranceSnapshots table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS TopInsuranceSnapshots (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                success INTEGER, -- SQLite uses 0 for False, 1 for True
                code TEXT,
                year INTEGER,
                period_id INTEGER, -- Month, quarter, or arbitrary ID
                source_file TEXT UNIQUE
            )
        ''')

        # 2. Create the TopInsuranceEntities table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS TopInsuranceEntities (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                snapshot_id INTEGER,
                entity_name TEXT,
                metric_type TEXT,
                metric_count INTEGER,
                metric_amount REAL,
                FOREIGN KEY (snapshot_id) REFERENCES TopInsuranceSnapshots(id)
            )
        ''')

        conn.commit()
        print("Database tables created/ensured for top insurance data.")
    except sqlite3.Error as e:
        print(f"Error setting up database: {e}")
    finally:
        if conn:
            conn.close()

# --- Data Insertion Function ---
def insert_top_insurance_snapshot(json_data, source_full_path, db_path):
    """
    Inserts data from a single parsed top insurance JSON object into the SQLite database.
    """
    conn = None
    source_filename = os.path.basename(source_full_path)
    
    # Extract year and period_id from the full file path
    year, period_id = extract_period_from_path(source_full_path)

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Extract data from the JSON object
        success = 1 if json_data.get("success") else 0 # Convert boolean to integer
        code = json_data.get("code")
        
        # Insert into TopInsuranceSnapshots table
        cursor.execute('''
            INSERT OR IGNORE INTO TopInsuranceSnapshots (
                success, code, year, period_id, source_file
            )
            VALUES (?, ?, ?, ?, ?)
        ''', (
            success, code, year, period_id, source_filename
        ))

        # If the snapshot was actually inserted, get its ID
        if cursor.rowcount > 0:
            snapshot_id = cursor.lastrowid
            
            # Process 'states' list
            states_list = json_data.get("data", {}).get("states", [])
            for entity_data in states_list:
                entity_name = entity_data.get("entityName")
                metric = entity_data.get("metric", {})
                metric_type = metric.get("type")
                metric_count = metric.get("count")
                metric_amount = metric.get("amount")

                cursor.execute('''
                    INSERT INTO TopInsuranceEntities (snapshot_id, entity_name, metric_type, metric_count, metric_amount)
                    VALUES (?, ?, ?, ?, ?)
                ''', (snapshot_id, entity_name, metric_type, metric_count, metric_amount))
            conn.commit()
            print(f"Successfully inserted data for snapshot '{source_filename}'.")
        else:
            print(f"Snapshot '{source_filename}' already exists in database. Skipping insertion.")

    except sqlite3.Error as e:
        print(f"Error inserting data from '{source_filename}': {e}")
        if conn:
            conn.rollback()
    except Exception as e: # Catch other potential errors during parsing/accessing
        print(f"General error processing '{source_filename}': {e}")
    finally:
        if conn:
            conn.close()

# --- Main Loading Logic ---
def load_folder_of_top_insurance_jsons_to_sqlite(folder_path, db_path):
    """
    Loads all JSON files from a specified folder and its subfolders (recursively)
    into a SQLite database for top insurance data.
    """
    setup_database(db_path) # Ensure tables are set up before starting insertions

    # Use glob to find all .json files in the folder and its subfolders
    json_files = glob.glob(os.path.join(folder_path, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"No JSON files found in '{folder_path}' or its subdirectories.")
        return

    print(f"Found {len(json_files)} JSON files. Starting insertion...")
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            insert_top_insurance_snapshot(data, json_file, db_path) # Pass full path for date extraction
        except FileNotFoundError:
            print(f"Error: JSON file not found (should not happen here): {json_file}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {json_file}. Skipping.")
        except PermissionError:
            print(f"Permission denied for file: {json_file}. Skipping.")
        except Exception as e:
            print(f"An unexpected error occurred while processing {json_file}: {e}")


# --- Execution ---
if __name__ == "__main__":
   

    load_folder_of_top_insurance_jsons_to_sqlite(json_root_dir, db_file_path)

    # Verify data
    print("\n--- Verifying loaded data ---")
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    cursor.execute("SELECT id, success, code, year, period_id, source_file FROM TopInsuranceSnapshots LIMIT 5")
    print("\nTopInsuranceSnapshots Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT snapshot_id, entity_name, metric_type, metric_count, metric_amount FROM TopInsuranceEntities LIMIT 5")
    print("\nTopInsuranceEntities Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT source_file, COUNT(*) FROM TopInsuranceSnapshots GROUP BY source_file")
    print("\nSnapshots loaded per source file:")
    for row in cursor.fetchall():
        print(row)
    
    cursor.execute("SELECT year, period_id, COUNT(*) FROM TopInsuranceSnapshots GROUP BY year, period_id ORDER BY year, period_id")
    print("\nSnapshots per Year and Period:")
    for row in cursor.fetchall():
        print(row)

    conn.close()

Database tables created/ensured for top insurance data.
Found 1036 JSON files. Starting insertion...
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in datab

In [181]:
import json

with open("C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\top\\user\\country\\india\\2018\\1.json", "r") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

{
    "success": true,
    "code": "SUCCESS",
    "data": {
        "states": [
            {
                "name": "maharashtra",
                "registeredUsers": 6106994
            },
            {
                "name": "uttar pradesh",
                "registeredUsers": 4694250
            },
            {
                "name": "karnataka",
                "registeredUsers": 3717763
            },
            {
                "name": "andhra pradesh",
                "registeredUsers": 3336450
            },
            {
                "name": "telangana",
                "registeredUsers": 3315560
            },
            {
                "name": "rajasthan",
                "registeredUsers": 3158202
            },
            {
                "name": "gujarat",
                "registeredUsers": 2690048
            },
            {
                "name": "west bengal",
                "registeredUsers": 2604789
            },
            {
                "name":

In [182]:

json_root_dir = "C:\\Users\\mukta\\OneDrive\\Desktop\\Phonepay\\pulse\\data\\top\\user\\country\\india" # <--- ADJUST THIS PATH

# Define the path for your SQLite database file
db_file_path = "top_user_data.db" # <--- NEW DATABASE NAME

# --- Utility Function to Extract Date/Period from Path ---
def extract_period_from_path(file_path):
    """
    Extracts year and a generic period ID from a file path using a common pattern like /YYYY/PERIOD.json
    Returns (year, period) or (None, None) if not found.
    """
    # Regex to find /YYYY/NUMBER.json pattern
    match = re.search(r'[\\/](\d{4})[\\/](\d{1,3})\.json$', file_path)
    if match:
        return int(match.group(1)), int(match.group(2))
    
    return None, None # Default if no pattern matches

# --- Database Setup Functions ---
def setup_database(db_path):
    """Sets up the SQLite database tables for top user data."""
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # 1. Create the TopUserSnapshots table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS TopUserSnapshots (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                success INTEGER, -- SQLite uses 0 for False, 1 for True
                code TEXT,
                year INTEGER,
                period_id INTEGER, -- Month, quarter, or arbitrary ID
                source_file TEXT UNIQUE
            )
        ''')

        # 2. Create the TopUserEntities table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS TopUserEntities (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                snapshot_id INTEGER,
                entity_name TEXT,
                registeredUsers INTEGER,
                FOREIGN KEY (snapshot_id) REFERENCES TopUserSnapshots(id)
            )
        ''')

        conn.commit()
        print("Database tables created/ensured for top user data.")
    except sqlite3.Error as e:
        print(f"Error setting up database: {e}")
    finally:
        if conn:
            conn.close()

# --- Data Insertion Function ---
def insert_top_user_snapshot(json_data, source_full_path, db_path):
    """
    Inserts data from a single parsed top user JSON object into the SQLite database.
    """
    conn = None
    source_filename = os.path.basename(source_full_path)
    
    # Extract year and period_id from the full file path
    year, period_id = extract_period_from_path(source_full_path)

    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Extract data from the JSON object
        success = 1 if json_data.get("success") else 0 # Convert boolean to integer
        code = json_data.get("code")
        
        # Insert into TopUserSnapshots table
        cursor.execute('''
            INSERT OR IGNORE INTO TopUserSnapshots (
                success, code, year, period_id, source_file
            )
            VALUES (?, ?, ?, ?, ?)
        ''', (
            success, code, year, period_id, source_filename
        ))

        # If the snapshot was actually inserted, get its ID
        if cursor.rowcount > 0:
            snapshot_id = cursor.lastrowid
            
            # Process 'states' list
            states_list = json_data.get("data", {}).get("states", [])
            for entity_data in states_list:
                entity_name = entity_data.get("name") # "name" for top user
                registered_users = entity_data.get("registeredUsers")

                cursor.execute('''
                    INSERT INTO TopUserEntities (snapshot_id, entity_name, registeredUsers)
                    VALUES (?, ?, ?)
                ''', (snapshot_id, entity_name, registered_users))
            conn.commit()
            print(f"Successfully inserted data for snapshot '{source_filename}'.")
        else:
            print(f"Snapshot '{source_filename}' already exists in database. Skipping insertion.")

    except sqlite3.Error as e:
        print(f"Error inserting data from '{source_filename}': {e}")
        if conn:
            conn.rollback()
    except Exception as e: # Catch other potential errors during parsing/accessing
        print(f"General error processing '{source_filename}': {e}")
    finally:
        if conn:
            conn.close()

# --- Main Loading Logic ---
def load_folder_of_top_user_jsons_to_sqlite(folder_path, db_path):
    """
    Loads all JSON files from a specified folder and its subfolders (recursively)
    into a SQLite database for top user data.
    """
    setup_database(db_path) # Ensure tables are set up before starting insertions

    # Use glob to find all .json files in the folder and its subfolders
    json_files = glob.glob(os.path.join(folder_path, '**', '*.json'), recursive=True)

    if not json_files:
        print(f"No JSON files found in '{folder_path}' or its subdirectories.")
        return

    print(f"Found {len(json_files)} JSON files. Starting insertion...")
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            insert_top_user_snapshot(data, json_file, db_path) # Pass full path for date extraction
        except FileNotFoundError:
            print(f"Error: JSON file not found (should not happen here): {json_file}")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {json_file}. Skipping.")
        except PermissionError:
            print(f"Permission denied for file: {json_file}. Skipping.")
        except Exception as e:
            print(f"An unexpected error occurred while processing {json_file}: {e}")


# --- Execution ---
if __name__ == "__main__":


    load_folder_of_top_user_jsons_to_sqlite(json_root_dir, db_file_path)

    # Verify data
    print("\n--- Verifying loaded data ---")
    conn = sqlite3.connect(db_file_path)
    cursor = conn.cursor()

    cursor.execute("SELECT id, success, code, year, period_id, source_file FROM TopUserSnapshots LIMIT 5")
    print("\nTopUserSnapshots Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT snapshot_id, entity_name, registeredUsers FROM TopUserEntities LIMIT 5")
    print("\nTopUserEntities Table (first 5 rows):")
    for row in cursor.fetchall():
        print(row)

    cursor.execute("SELECT source_file, COUNT(*) FROM TopUserSnapshots GROUP BY source_file")
    print("\nSnapshots loaded per source file:")
    for row in cursor.fetchall():
        print(row)
    
    cursor.execute("SELECT year, period_id, COUNT(*) FROM TopUserSnapshots GROUP BY year, period_id ORDER BY year, period_id")
    print("\nSnapshots per Year and Period:")
    for row in cursor.fetchall():
        print(row)

    conn.close()

Database tables created/ensured for top user data.
Found 1036 JSON files. Starting insertion...
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. Skipping insertion.
Snapshot '3.json' already exists in database. Skipping insertion.
Snapshot '4.json' already exists in database. Skipping insertion.
Snapshot '1.json' already exists in database. Skipping insertion.
Snapshot '2.json' already exists in database. 

In [188]:
import sqlite3

DB_PATH = 'ag_insurance_data.db'

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")


Tables found in 'ag_insurance_data.db':
- Transactions
- sqlite_sequence
- PaymentInstruments

Tables found in 'ag_insurance_data.db':
- Transactions
- sqlite_sequence
- PaymentInstruments


In [189]:
import sqlite3

DB_PATH = 'top_insurance_data.db'

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")


Tables found in 'top_insurance_data.db':
- TopInsuranceSnapshots
- sqlite_sequence
- TopInsuranceEntities

Tables found in 'top_insurance_data.db':
- TopInsuranceSnapshots
- sqlite_sequence
- TopInsuranceEntities


In [190]:
import sqlite3

DB_PATH = 'ag_transaction_data.db'

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")


Tables found in 'ag_transaction_data.db':
- Transactions
- sqlite_sequence
- PaymentInstruments

Tables found in 'ag_transaction_data.db':
- Transactions
- sqlite_sequence
- PaymentInstruments


In [191]:
import sqlite3

DB_PATH = 'ag_user_data.db'

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")


Tables found in 'ag_user_data.db':
- Transactions
- sqlite_sequence
- PaymentInstruments
- AggregatedSnapshots
- UsersByDevice

Tables found in 'ag_user_data.db':
- Transactions
- sqlite_sequence
- PaymentInstruments
- AggregatedSnapshots
- UsersByDevice


In [192]:
import sqlite3

DB_PATH = 'map_insurance_data.db'

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")


Tables found in 'map_insurance_data.db':
- MapInsuranceSnapshots
- sqlite_sequence
- StateMetrics
- DistrictMetrics

Tables found in 'map_insurance_data.db':
- MapInsuranceSnapshots
- sqlite_sequence
- StateMetrics
- DistrictMetrics


In [193]:
import sqlite3

DB_PATH = 'map_user_data.db'

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")


Tables found in 'map_user_data.db':
- MapHoverSnapshots
- sqlite_sequence
- HoverDataEntries

Tables found in 'map_user_data.db':
- MapHoverSnapshots
- sqlite_sequence
- HoverDataEntries


In [194]:
import sqlite3

DB_PATH = 'map_hover_data.db'

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")


Tables found in 'map_hover_data.db':
- MapHoverSnapshots
- sqlite_sequence
- HoverDataEntries

Tables found in 'map_hover_data.db':
- MapHoverSnapshots
- sqlite_sequence
- HoverDataEntries


In [195]:
import sqlite3

DB_PATH = 'map_transaction_data.db'

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")


Tables found in 'map_transaction_data.db':
- MapHoverSnapshots
- sqlite_sequence
- HoverDataEntries

Tables found in 'map_transaction_data.db':
- MapHoverSnapshots
- sqlite_sequence
- HoverDataEntries


In [196]:
import sqlite3

DB_PATH = 'top_transaction_data.db'

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")


Tables found in 'top_transaction_data.db':
- TopInsuranceSnapshots
- sqlite_sequence
- TopInsuranceEntities

Tables found in 'top_transaction_data.db':
- TopInsuranceSnapshots
- sqlite_sequence
- TopInsuranceEntities


In [197]:
import sqlite3

DB_PATH = 'top_user_data.db'

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")

if os.path.exists(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        conn.close()

        if tables:
            print(f"\nTables found in '{DB_PATH}':")
            for table in tables:
                print(f"- {table[0]}")
        else:
            print(f"\nNo tables found in '{DB_PATH}'. The database might be empty.")

    except sqlite3.Error as e:
        print(f"Error connecting to database or listing tables: {e}")
else:
    print(f"Cannot list tables, as the database file '{DB_PATH}' does not exist.")


Tables found in 'top_user_data.db':
- TopUserSnapshots
- sqlite_sequence
- TopUserEntities

Tables found in 'top_user_data.db':
- TopUserSnapshots
- sqlite_sequence
- TopUserEntities


In [198]:
import sqlite3
import pandas as pd
import os

# --- Function to run a query and display results ---
def run_query(db_path, query_name, query):
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        print(f"\n--- Running Query: {query_name} on {db_path} ---")
        df = pd.read_sql_query(query, conn)
        if df.empty:
            print("No data found for this query.")
        else:
            display(df) 
    except sqlite3.Error as e:
        print(f"Error executing query on {db_path}: {e}")
    finally:
        if conn:
            conn.close()



DB_PATH = 'top_transaction_data.db'

In [ ]:
# Query 1: Total Insurance Trends Over Time (Year and Quarter)
query_name_1 = "Total Insurance Trends by Year and Quarter"
query_1 = """
SELECT
    TIS.year,
    TIS.period_id AS quarter,
    SUM(TIE.metric_count) AS total_insurance_count,
    SUM(TIE.metric_amount) AS total_insurance_amount
FROM
    TopInsuranceSnapshots AS TIS
JOIN
    TopInsuranceEntities AS TIE
ON
    TIS.id = TIE.snapshot_id
GROUP BY
    TIS.year,
    TIS.period_id
ORDER BY
    TIS.year,
    TIS.period_id;
"""

run_query(DB_PATH, query_name_1, query_1)

In [ ]:
# Query 2: Top 10 States by Total Insurance Amount
query_name_2_amount = "Top 10 States by Total Insurance Amount"
query_2_amount = """
SELECT
    TIE.entity_name AS state,
    SUM(TIE.metric_amount) AS total_insurance_amount,
    SUM(TIE.metric_count) AS total_insurance_count
FROM
    TopInsuranceEntities AS TIE
GROUP BY
    TIE.entity_name
ORDER BY
    total_insurance_amount DESC
LIMIT 10;
"""

run_query(DB_PATH, query_name_2_amount, query_2_amount)



In [ ]:
# Query 3: Quarterly Growth Rate of Insurance Amount by State (Example for 2023)
query_name_3 = "Quarterly Insurance Amount Growth Rate by State (2023)"
query_3 = """
WITH QuarterlyData AS (
    SELECT
        TIS.year,
        TIS.period_id AS quarter,
        TIE.entity_name AS state,
        SUM(TIE.metric_amount) AS quarterly_amount
    FROM
        TopInsuranceSnapshots AS TIS
    JOIN
        TopInsuranceEntities AS TIE
    ON
        TIS.id = TIE.snapshot_id
    WHERE
        TIS.year = 2023 -- Focusing on a specific year for quarterly comparison
    GROUP BY
        TIS.year,
        TIS.period_id,
        TIE.entity_name
),
LaggedData AS (
    SELECT
        year,
        quarter,
        state,
        quarterly_amount,
        LAG(quarterly_amount, 1, 0) OVER (PARTITION BY state ORDER BY year, quarter) AS previous_quarter_amount
    FROM
        QuarterlyData
)
SELECT
    year,
    quarter,
    state,
    quarterly_amount,
    previous_quarter_amount,
    CASE
        WHEN previous_quarter_amount > 0 THEN
            ROUND(((quarterly_amount - previous_quarter_amount) * 100.0 / previous_quarter_amount), 2)
        ELSE 0 -- Handle cases where previous quarter amount is zero or null
    END AS growth_rate_percentage
FROM
    LaggedData
ORDER BY
    state,
    year,
    quarter;
"""

run_query(DB_PATH, query_name_3, query_3)

In [ ]:
5] python data analysis

In [ ]:
# Handling missing values
def clean_data(df):
    # Fill numeric missing values with median
    numeric_cols = df.select_dtypes(include=['number']).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
    
    # Fill categorical missing values with mode
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode()[0])
    
    # Convert date columns if they exist
    date_cols = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    return df


In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
def analyze_top_user_data():
    conn = sqlite3.connect('top_user_data.db')
    
    # Load TopUserSnapshots data
    snapshots = pd.read_sql("SELECT * FROM TopUserSnapshots", conn)
    
    # Load TopUserEntities data
    entities = pd.read_sql("SELECT * FROM TopUserEntities", conn)
    
    # Basic analysis
    print("Top User Snapshots Summary:")
    print(snapshots.describe())
    print("\nTop User Entities Summary:")
    print(entities.describe())
    
    # Visualization - Top Users over time
    if 'timestamp' in snapshots.columns and 'user_count' in snapshots.columns:
        snapshots['timestamp'] = pd.to_datetime(snapshots['timestamp'])
        snapshots.set_index('timestamp')['user_count'].plot(title='Top Users Over Time')
        plt.ylabel('User Count')
        plt.show()
    
    # Visualization - Top User Distribution
    if not entities.empty:
        if 'user_name' in entities.columns and 'value' in entities.columns:
            entities.nlargest(10, 'value').plot.bar(x='user_name', y='value', 
                                                   title='Top 10 Users by Metric')
            plt.ylabel('Metric Value')
            plt.show()
    
    conn.close()

analyze_top_user_data()

In [ ]:
def analyze_top_transaction_data():
    conn = sqlite3.connect('top_transaction_data.db')
    
    # Load data
    snapshots = pd.read_sql("SELECT * FROM TopInsuranceSnapshots", conn)
    entities = pd.read_sql("SELECT * FROM TopInsuranceEntities", conn)
    
    # Basic analysis
    print("Top Insurance Snapshots Summary:")
    print(snapshots.describe())
    print("\nTop Insurance Entities Summary:")
    print(entities.describe())
    
    # Visualization - Insurance trends over time
    if 'timestamp' in snapshots.columns and 'transaction_count' in snapshots.columns:
        snapshots['timestamp'] = pd.to_datetime(snapshots['timestamp'])
        snapshots.set_index('timestamp')['transaction_count'].plot(
            title='Insurance Transactions Over Time')
        plt.ylabel('Transaction Count')
        plt.show()
    
    # Visualization - Top Insurance Entities
    if not entities.empty and 'entity_name' in entities.columns and 'value' in entities.columns:
        entities.nlargest(10, 'value').plot.bar(x='entity_name', y='value',
                                               title='Top 10 Insurance Entities')
        plt.ylabel('Metric Value')
        plt.show()
    
    conn.close()

analyze_top_transaction_data()

In [ ]:
def analyze_map_insurance_data():
    conn = sqlite3.connect('map_insurance_data.db')
    
    # Load data
    snapshots = pd.read_sql("SELECT * FROM MapInsuranceSnapshots", conn)
    state_metrics = pd.read_sql("SELECT * FROM StateMetrics", conn)
    district_metrics = pd.read_sql("SELECT * FROM DistrictMetrics", conn)
    
    # Basic analysis
    print("State Metrics Summary:")
    print(state_metrics.describe())
    print("\nDistrict Metrics Summary:")
    print(district_metrics.describe())
    
    # Visualization - State-level metrics
    if not state_metrics.empty and 'state_name' in state_metrics.columns:
        if 'total_transactions' in state_metrics.columns:
            state_metrics.nlargest(10, 'total_transactions').plot.bar(
                x='state_name', y='total_transactions',
                title='Top 10 States by Insurance Transactions')
            plt.ylabel('Transaction Count')
            plt.show()
        
        if 'average_value' in state_metrics.columns:
            state_metrics.nlargest(10, 'average_value').plot.bar(
                x='state_name', y='average_value',
                title='Top 10 States by Average Insurance Value')
            plt.ylabel('Average Value')
            plt.show()
    
    # Visualization - District-level metrics (sample)
    if not district_metrics.empty and 'district_name' in district_metrics.columns:
        top_districts = district_metrics.nlargest(10, 'total_transactions')
        plt.figure(figsize=(12, 6))
        sns.barplot(data=top_districts, x='district_name', y='total_transactions',
                   hue='state_name', dodge=False)
        plt.title('Top 10 Districts by Insurance Transactions')
        plt.xticks(rotation=45)
        plt.ylabel('Transaction Count')
        plt.show()
    
    conn.close()

analyze_map_insurance_data()

In [ ]:
def analyze_ag_user_data():
    conn = sqlite3.connect('ag_user_data.db')
    
    # Load data
    transactions = pd.read_sql("SELECT * FROM Transactions", conn)
    payment_instruments = pd.read_sql("SELECT * FROM PaymentInstruments", conn)
    snapshots = pd.read_sql("SELECT * FROM AggregatedSnapshots", conn)
    users_by_device = pd.read_sql("SELECT * FROM UsersByDevice", conn)
    
    # Basic analysis
    print("Transactions Summary:")
    print(transactions.describe())
    print("\nPayment Instruments Summary:")
    print(payment_instruments.describe())
    
    # Visualization - Payment instrument distribution
    if not payment_instruments.empty and 'instrument_type' in payment_instruments.columns:
        payment_instruments['instrument_type'].value_counts().plot.pie(
            autopct='%1.1f%%', title='Payment Instrument Distribution')
        plt.ylabel('')
        plt.show()
    
    # Visualization - Users by device
    if not users_by_device.empty and 'device_type' in users_by_device.columns:
        users_by_device.plot.bar(x='device_type', y='user_count',
                               title='Users by Device Type')
        plt.ylabel('User Count')
        plt.show()
    
    # Visualization - Transaction trends over time
    if not snapshots.empty and 'timestamp' in snapshots.columns:
        snapshots['timestamp'] = pd.to_datetime(snapshots['timestamp'])
        snapshots.set_index('timestamp')[['transaction_count', 'user_count']].plot(
            title='Transaction and User Trends Over Time')
        plt.ylabel('Count')
        plt.show()
    
    conn.close()

analyze_ag_user_data()

In [ ]:
def analyze_map_hover_data():
    conn = sqlite3.connect('map_hover_data.db')
    
    # Load data
    snapshots = pd.read_sql("SELECT * FROM MapHoverSnapshots", conn)
    hover_data = pd.read_sql("SELECT * FROM HoverDataEntries", conn)
    
    # Basic analysis
    print("Hover Data Summary:")
    print(hover_data.describe())
    
    # Visualization - Hover events over time
    if not snapshots.empty and 'timestamp' in snapshots.columns:
        snapshots['timestamp'] = pd.to_datetime(snapshots['timestamp'])
        snapshots.set_index('timestamp')['hover_count'].plot(
            title='Map Hover Events Over Time')
        plt.ylabel('Hover Count')
        plt.show()
    
    # Visualization - Hover data by region
    if not hover_data.empty and 'region_name' in hover_data.columns:
        hover_data.nlargest(10, 'hover_count').plot.bar(
            x='region_name', y='hover_count',
            title='Top 10 Regions by Hover Events')
        plt.ylabel('Hover Count')
        plt.show()
    
    conn.close()

analyze_map_hover_data()

In [ ]:
3. Marketing & Product Recommendations

A. Targeted Campaigns
Offer Cashback for UPI transactions in Tier-2 cities to boost adoption.
Promote Insurance Bundles during tax season (January–March).

B. Product Improvements
Simplify KYC for rural users to reduce drop-offs.
Introduce SIP-like Insurance Plans for young users.